# Random Walks en Multiplex de Aerolíneas

## Directorio

In [1]:
#Establecer directorio
cd("$(homedir())/Documents/UNAM/Tesis_Lic")

## Paquetes

Se importan todos los paquetes que se necesitarán para las funciones y visualizaciones del proyecto.

In [2]:
using StatsBase

#using NamedArrays
#using CSV
#using DataFrames

In [3]:
using DelimitedFiles

In [4]:
using RandomNumbers.Xorshifts

In [5]:
#using LightGraphs

In [6]:
#using Plots

In [7]:
#using GraphPlot

In [8]:
#Leo archivo de diccionario final
#archivo_dicc = readdlm("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/diccionarios/dictionary_final.csv", ',') 

## Funciones

### Importar datos

In [9]:
info_aerolineas = readdlm("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/aerolineas_matrices/nueva_lista_aerolineas.csv", ',', String)
lista_aerolineas = info_aerolineas[:, 2]

568-element Array{String,1}:
 "Airlines"
 "US"      
 "AA"      
 "TK"      
 "DL"      
 "MU"      
 "CZ"      
 "UA"      
 "FR"      
 "BA"      
 "AF"      
 "LH"      
 "CA"      
 ⋮         
 "GY"      
 "EW"      
 "D6"      
 "9D"      
 "7E"      
 "6Y"      
 "6I"      
 "5M"      
 "5G"      
 "4K"      
 "4D"      
 "4B"      

In [10]:
i = 2
airline_code = lista_aerolineas[i]

"US"

In [11]:
aerolinea_matriz = string(airline_code, "_matriz_du.csv")

"US_matriz_du.csv"

In [12]:
archivo_dicc = readdlm("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/diccionarios/dictionary_final.csv", ',')

3190×570 Array{Any,2}:
     "line"  "global_airports"  …  "ZK_num"  "ZL_num"   "ZM_num"
    1        "AER"                 "NA"      "NA"       "NA"    
    2        "ASF"                 "NA"      "NA"       "NA"    
    3        "CEK"                 "NA"      "NA"       "NA"    
    4        "DME"                 "NA"      "NA"      1        
    5        "EGO"              …  "NA"      "NA"       "NA"    
    6        "GYD"                 "NA"      "NA"       "NA"    
    7        "KGD"                 "NA"      "NA"       "NA"    
    8        "KZN"                 "NA"      "NA"       "NA"    
    9        "LED"                 "NA"      "NA"       "NA"    
   10        "MRV"              …  "NA"      "NA"       "NA"    
   11        "NBC"                 "NA"      "NA"       "NA"    
   12        "NJC"                 "NA"      "NA"       "NA"    
    ⋮                           ⋱                               
 3178        "LPS"                 "NA"      "NA"       "NA"    
 3

In [13]:
"""
    import_matrix(arch::AbstractString)

Función que importa el archivo de la matriz de adyacencia de R.
"""
function import_matrix(arch::AbstractString)
    archivo_R = readdlm(arch, ',') 
    archivo_R = archivo_R[2:end, :]
    matriz = archivo_R[:, 2:end]
    nombre = convert(Array{Int64,2}, matriz)
    return nombre
end

import_matrix

In [14]:
multiplex = @time import_matrix("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/aerolineas_matrices/$aerolinea_matriz")

  0.192376 seconds (625.41 k allocations: 28.694 MiB)


280×280 Array{Int64,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  

### Nombres de datos

In [15]:
"""
    airport_names(arch::AbstractString)

Función que obtiene un arreglo con los nombres de aeropuertos de la matriz
"""
function airport_names(arch::AbstractString)
    archivo_R = readdlm(arch, ',', String)
    columna = archivo_R[1, :]
    nombres = columna[2:end]
    return nombres
end

airport_names

In [16]:
data_names_multiplex = @time airport_names("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/aerolineas_matrices/$aerolinea_matriz")

  0.133042 seconds (649.56 k allocations: 28.936 MiB)


280-element Array{String,1}:
 "ABI"
 "ABQ"
 "ACT"
 "ALB"
 "ALO"
 "AMA"
 "AMM"
 "AMS"
 "ANC"
 "ANU"
 "ART"
 "ASU"
 "ATH"
 ⋮    
 "AGS"
 "CRW"
 "LYH"
 "PWM"
 "AEX"
 "AGU"
 "GCK"
 "GJT"
 "MGM"
 "SLP"
 "SPI"
 "YKF"

In [17]:
#data_names_multiplex[65]

### Grado

### Vecinos

In [18]:
"""
    neighbours(M::Array{Int64,2})

Función que obtiene un conjunto de arreglos con los vecinos de cada nodo.
"""
function neighbours(M::Array{Int64,2})
    
    nodos = size(M, 1) #aeropuertos
    vecindades = []
    
    for i in 1:nodos
        vecinos = findall(x -> x != 0, M[i, :])
        push!(vecindades, vecinos)
    end
    vecindades
end

neighbours

In [19]:
neighbors_multiplex = @time neighbours(multiplex);

  0.090635 seconds (206.23 k allocations: 10.551 MiB)


### Random walks

In [20]:
"""

    r_walks(M::Array{Int64,2}, f::Int64, steps::Int64, iter::Int64)

La función r_walks tiene como parámetros:
M, matriz de adyacencia de tipo Array{Int64,2}
f, nodo inicial de tipo Int64
steps, número de pasos de tipo Int64
iter, número de iteraciones de tipo Int64.
La función crea un caminante aleatorio sobre la matriz, el cual tiene igual probabilidad de avanzar 
hacia cualquiera de sus vecinos.
Regresa un arreglo que contiene todas las trayectorias que tomó el caminante aleatorio.
"""
function r_walks(M::Array{Int64,2}, f::Int64, steps::Int64, iter::Int64)
    
    paths = zeros(Int64, (iter, steps + 1))
    s = f
    seed = 1
    vecindades = neighbours(M)

    for i in 1:iter
        paths[i, 1] = f
        
        for p in 1:steps 
            vecindad = vecindades[s]
            
            if vecindad == []
                break
            else
                a = sample(Xoroshiro128Plus(seed), vecindad) 
                paths[i, p + 1] = a
                s = a
                seed += 1
            end
            
        end
        s = f
        
    end
    
    return paths 
end

r_walks

In [21]:
"""

    random_walk(M::Array{Int64,2}, steps::Int64, iter::Int64)

La función random_walk tiene como parámetros:
M, matriz de adyacencia de tipo Array{Int64,2}
steps, número de pasos de tipo Int64
iter, número de iteraciones de tipo Int64.
La función crea un caminante aleatorio sobre la matriz con igual probabilidad de avanzar 
hacia cualquiera de sus vecinos. Esta función recorre todos los nodos de la red como
nodos iniciales y sobre ellos itera para obtener el número de caminos que se le pidió.
Regresa un arreglo que contiene arreglos asociados a todas las trayectorias que tomó el 
caminante aleatorio para cada nodo inicial de la red.
"""
function random_walk(M::Array{Int64,2}, steps::Int64, iter::Int64) 
    
    random_walks = []
    nodos = size(M, 1)
    
    for f in 1:nodos #aeropuertos
        paths = r_walks(M, f, steps, iter)
        push!(random_walks, paths)
    end
    
    random_walks
end

random_walk

In [22]:
random_walk_multiplex = @time random_walk(multiplex, 50, 2)

  0.883565 seconds (1.07 M allocations: 230.855 MiB, 65.09% gc time)


280-element Array{Any,1}:
 [1 65 … 0 0; 1 65 … 95 47]     
 [2 180 … 0 0; 2 180 … 95 47]   
 [3 65 … 0 0; 3 65 … 95 47]     
 [4 63 … 0 0; 4 63 … 95 47]     
 [5 180 … 0 0; 5 180 … 95 47]   
 [6 65 … 0 0; 6 65 … 95 47]     
 [7 263 … 0 0; 7 180 … 95 47]   
 [8 189 … 0 0; 8 189 … 0 0]     
 [9 190 … 0 0; 9 190 … 0 0]     
 [10 156 … 0 0; 10 117 … 95 47] 
 [11 180 … 0 0; 11 180 … 95 47] 
 [12 156 … 0 0; 12 156 … 117 79]
 [13 189 … 0 0; 13 189 … 0 0]   
 ⋮                              
 [269 0 … 0 0; 269 0 … 0 0]     
 [270 0 … 0 0; 270 0 … 0 0]     
 [271 0 … 0 0; 271 0 … 0 0]     
 [272 0 … 0 0; 272 0 … 0 0]     
 [273 0 … 0 0; 273 0 … 0 0]     
 [274 0 … 0 0; 274 0 … 0 0]     
 [275 0 … 0 0; 275 0 … 0 0]     
 [276 0 … 0 0; 276 0 … 0 0]     
 [277 0 … 0 0; 277 0 … 0 0]     
 [278 0 … 0 0; 278 0 … 0 0]     
 [279 0 … 0 0; 279 0 … 0 0]     
 [280 0 … 0 0; 280 0 … 0 0]     

In [23]:
random_walk_multiplex[1]

2×51 Array{Int64,2}:
 1  65  253   65  219   65  189  258  …   47  10  156  266  65  276   0   0
 1  65  238  190   36  156  221   63     189  81   27  117  79   47  95  47

### Nodos/aeropuertos visitados

In [24]:
"""
    visited_vertices(W::Array{Any,1})

La función caminos tiene como parámetro W que es un arreglo de arreglos.
Regresa los nodos diferentes a los que llega el caminante.
"""
function visited_vertices(W::Array{Any,1})
    
    cobert = Array[]
    v = length(W) #tamaño de la matriz, número de nodos
    s = size(W[1], 1) #lo puedo sacar, para todos es el mismo. Es el número de iteraciones
    t = size(W[1], 2) #número de pasos, no necesario
    #visit = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    visit = Array[]
    
    for i in 1:v        
        for j in 1:s
            u = unique(W[i][j, :])
            push!(visit, u)
        end
        
        push!(cobert, visit)
        visit = []
    end
    
    return cobert
end

visited_vertices

In [25]:
visited_vertices(random_walk_multiplex)[1]

2-element Array{Array,1}:
 [1, 65, 253, 219, 189, 258, 184, 112, 156, 40  …  202, 122, 117, 216, 228, 187, 10, 266, 276, 0]
 [1, 65, 238, 190, 36, 156, 221, 63, 205, 122  …  83, 109, 224, 102, 8, 144, 81, 27, 79, 95]     

### Random walks con nodos globales

In [26]:
"""
    walk_global(W::Array{Any,1}, airline_code::String, v::Int64)

Función que cambia las caminatas del número de nodo asociado a la aerolínea a número de nodo GLOBAL de aeropuertos.
Parámetros:
W = caminatas aleatorias (arreglo de matrices)
airline_code = código de la aerolínea
v = nodo inicial
"""
function walk_global(W::Array{Any,1}, airline_code::String, v::Int64)
    
    #Ahora se usa la función para obtener los nodos diferentes
    nodos_diferentes = visited_vertices(W)
    
    #vv2b[1][1,:] #Nodos únicos visitados en la iteración 1 con nodo inicial 1 #Leo los visited vertices del archivo
    m = size(W[1], 1) #renglones/iteraciones
    #@show m
    n = size(W[1], 2) #columnas/pasos+1/nodos
    #@show n
    l = length(W) #número de nodos iniciales
    #@show l
    
    #Se lee el diccionario
     #archivo_dicc
    #@show archivo_dicc
    aerol_col = string(airline_code, "_num") #Nombre de la columna
    #@show aerol_col
    coord_aerol = findall(x -> x == aerol_col, archivo_dicc) #Busco las coordenadas de la columna
    #@show coord_aerol
    col_aerol = coord_aerol[1][2] #Segunda coordenada de la posición (columna)
    columna_aerol = archivo_dicc[:, col_aerol] #Obtengo la columna con el nombre del aeropuerto al que corresponde
    #@show columna_aerol
    walk_empty = zeros(Int64, m, n)#Arreglo vacío para nuevos números
        #@show walk_empty
    
    for k = v #1:l #nodo inicial v
        #@show k
        arreglo_visitas = nodos_diferentes[k]
        #@show arreglo_visitas
        caminatas_nodo = W[k] #caminata y sus iteraciones con el mismo nodo inicial
        #@show caminatas_nodo
        
        
        for i in 1:m #voy por las iteraciones
            #@show i
            nodos_dif = arreglo_visitas[i, :] #array/caminata única de los nodos que buscará
            #@show nodos_dif
            
            for j in 1:length(nodos_dif[]) #vamos por los nodos diferentes por los que pasé
                #@show j
                
                nodo_original = nodos_dif[][j] #index original de la aerolínea
                #@show nodo_original
                
                #Busco la posición original del aeropuerto que se asignará
               # if 
                posicion_original = findall(x -> x == nodo_original, archivo_dicc[:, col_aerol]) 
                #Este arreglo tendrá la coordenada de la posición del nodo inicial, sin embargo, si llegué a un endpoint, 
                #buscará la coordenada de la posición del nodo 0 y te regresará un un arreglo vacío.
                #@show posicion_original
                coords_ys = findall(x -> x == nodo_original, caminatas_nodo[i, :]) #Encuentra uno de los nodos dif en la iteración
                        #@show coords_ys
                
                #Se busca el nombre del aeropuerto
                
                for q in 1:length(coords_ys) #Se itera sobre las posiciones en las caminatas
                        #@show length(coords_ys)
                        #@show q
                        
                        pos_actual = coords_ys[q]
                
                        if isempty(posicion_original) == false
                    
                            numero_asignado = archivo_dicc[posicion_original, 3]
                            #@show numero_asignado
                            #Busco el primer nodo diferente en la iteración i
                            
                            
                            walk_empty[i, pos_actual] = numero_asignado[] #asigno valor de diccionario al arreglo vacío i, pos_actual
                            #@show walk_empty
                    
                
                
                        else break
                    end
                
                end
                
            end
            #@show walk_empty

        end
        #@show walk_empty
        
        
    end
    
    return walk_empty
   
end

walk_global

In [27]:
for v = 1
    walk_global_multiplex = @time walk_global(random_walk_multiplex, airline_code, v)
    return walk_global_multiplex
end

  0.590648 seconds (6.54 M allocations: 266.688 MiB, 6.90% gc time)


2×51 Array{Int64,2}:
 2875  1130  1211  1130  1197  1130  …  369  1024  1130  2046     0    0
 2875  1130   124  1185  1115   369      44   393   451   715  1793  715

In [28]:
function walks_global(W::Array{Any,1}, airline_code::String)
    
    t = length(W)
    #@show t
    
    arreglo = Array[]
    
    for v in 267:t
        #@show v
        #archivo_caminata_global = string(airline_code, "_", v,"_walk_global.csv")
        #@show archivo_caminata_global
        
        walk_global_multiplex = walk_global(W, airline_code, v)
        #return walk_global_multiplex
        push!(arreglo, walk_global_multiplex)
        #writedlm("/Users/Yuriko/Desktop/carpetita/$archivo_caminata_global", walk_global_multiplex, ',')
    end
    return arreglo
end

walks_global (generic function with 1 method)

In [29]:
#walks_global(random_walk_multiplex, airline_code)

In [30]:
#for v = 1:length(random_walk_multiplex)
#    @show v
#    archivo_caminata_global = string(airline_code, "_", v,"_walk_global.csv")
#    @show archivo_caminata_global
#	walk_global_multiplex = @time walk_global(random_walk_multiplex, airline_code, v)
#    return walk_global_multiplex
#    writedlm("/Users/Yuriko/Desktop/carpetita/$archivo_caminata_global", walk_global_multiplex, ',')
#end

### Random walks con nombres

In [31]:
"""
    walk_names(W::Array{Any,1}, airline_code::String, v::Int64)

Función que cambia las caminatas del número de nodo asociado a la aerolínea a nombres de aeropuertos.
Parámetros:
W = caminatas aleatorias (arreglo de matrices)
airline_code = código de la aerolínea
v = nodo inicial
"""
function walk_names(W::Array{Any,1}, airline_code::String, v::Int64, dicc) 
    
    #Ahora se usa la función para obtener los nodos diferentes
    nodos_diferentes = visited_vertices(W)
    m = size(W[1], 1) #renglones
    n = size(W[1], 2) #columnas
    l = length(W) #número de nodos iniciales
    #ep = ["Endpoint"]
    
    #Se lee el diccionario
    #archivo_dicc = readdlm("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/diccionarios/dictionary_final.csv", ',') 
    archivo_dicc = dicc
    
    aerol_col = string(airline_code, "_num") #Nombre de la columna
    coord_aerol = findall(x -> x == aerol_col, archivo_dicc) #Busco las coordenadas de la columna
    col_aerol = coord_aerol[1][2] #Segunda coordenada de la posición (columna)
    columna_aerol = archivo_dicc[:, col_aerol] #Obtengo la columna con el nombre del aeropuerto al que corresponde
    walk_nothing = Array{Any}(undef, m, n)#Array{Union{Nothing, String}}(nothing, m, n) #archivo vacío de una matriz para un nodo inicial
    #@show walk_nothing
    
    for k = v #nodo inicial v
        arreglo_visitas = nodos_diferentes[k]
        caminatas_nodo = W[k]
        
        for i in 1:m #voy por las iteraciones
            nodos_dif = arreglo_visitas[i, :] #array/caminata única de los nodos que buscará
            
            for j in 1:length(nodos_dif[]) #para cada uno de los nodos diferentes visitados
                nodo_original = nodos_dif[][j] #tomo el nodo original visitado
                
                #Busco la posición original del aeropuerto visitado en la columna de la aerolínea
                posicion_original = findall(x -> x == nodo_original, archivo_dicc[:, col_aerol])
                
                #Ahora busco dónde se colocará el nombre en las posiciones correctas del arreglo vacío
                #Busco el primer nodo diferente en la iteración i
                #Encuentra todas las coordenadas de los nodos dif en la iteración original
                coords_ys = findall(x -> x == nodo_original, caminatas_nodo[i, :]) 
            
                
                for q in 1:length(coords_ys) #Se itera sobre las posiciones en las caminatas
                    
                    pos_actual = coords_ys[q]
                
                    if isempty(posicion_original) == false
                        
                        #Se busca el nombre del aeropuerto
                        nombre_asignado = archivo_dicc[posicion_original, 2] #y=2 porque esa es la columna de los nombres
                    
                        walk_nothing[i, pos_actual] = nombre_asignado[] #asigno valor de diccionario al arreglo vacío i, pos_actual

                        else walk_nothing[i, pos_actual] = " " #break
                    end
                end
                
            end

        end
        
    end
    return walk_nothing
end

walk_names

In [32]:
for v = 1:2#:length(random_walk_multiplex)
	walk_names_multiplex = @time walk_names(random_walk_multiplex, airline_code, v, archivo_dicc)
    walk_names_multiplex
    archivo_caminata_nombres = string(airline_code, "_", v,"_walk_names.csv")
    writedlm("/Users/yuriko/Desktop/$archivo_caminata_nombres", walk_names_multiplex, ',')
end

  0.464769 seconds (6.22 M allocations: 250.798 MiB, 7.71% gc time)
  0.277751 seconds (5.87 M allocations: 234.812 MiB, 12.63% gc time)
